# Load and parse indri files

In [1]:
import os

Meta1Resultspath='/home/ubuntu/SpotifyChallenge/notebooks/Reranking/TestingQueryResults/Meta1/'
Meta2Resultspath='/home/ubuntu/SpotifyChallenge/notebooks/Reranking/TestingQueryResults/Meta2/'
QEPRFResultspath='/home/ubuntu/SpotifyChallenge/notebooks/Reranking/TestingQueryResults/QEPRF750/'

Meta1Files=[Meta1Resultspath+x for x in os.listdir(Meta1Resultspath)]
Meta2Files=[Meta2Resultspath+x for x in os.listdir(Meta2Resultspath)]
QEPRFFiles=[QEPRFResultspath+x for x in os.listdir(QEPRFResultspath)]

In [2]:
#parse QE-PRF Indri files
import codecs
def QEPRFParse(path):
    playlistId=path.split('/')[-1].split('.op')[0]
    with codecs.open(path, 'r', encoding='utf-8') as f:
        lines = f.read().splitlines()
    inputQueries=lines[0].split('# query: ')[1].split()
    return(playlistId,inputQueries)

In [3]:
QEPRFOp={}
err3=[]
for f in QEPRFFiles:
    res=QEPRFParse(f)
    if res !="QueryError":
        QEPRFOp[res[0]]=res[1]
    else:
        err3.append(f)

In [4]:
#load mapping of pid and tracks
import pickle
trackFeatures=pickle.load(open('/home/ubuntu/GetTrackData/trackFeatures.pkl','rb'))
trackDetails=pickle.load(open('/home/ubuntu/GetTrackData/trackDetails.pkl','rb'))


In [5]:
for i in trackFeatures:
    print trackFeatures[i]
    break
for i in trackDetails:
    print trackDetails[i]
    break

[9, 0.858, 0.159, 115.004, 0.0408, 0.0532, 0.722, -4.213, 0.753, 8.69e-06]
[u'2013-03-12', 2, False, u'US9N71300007']


In [6]:
len(QEPRFOp), len(trackFeatures), len(trackDetails)

(4500, 2262133, 2261675)

In [7]:
#QEPRFOp format: playlistId mapped to [reference set(queries), predictions(rank, playlistid, score)]
QEPRFOp.items()[0]

('503643',
 [u'6O6M7pJLABmfBRoGZMu76Y',
  u'1wYZZtamWTQAoj8B812uKQ',
  u'539dFCMA8lfEElDPgGpcIB',
  u'3WKg25vrbjJlkhsgl2W4p3',
  u'3Qe097eJNp9MfpT2PieEw0',
  u'2GiJYvgVaD2HtM8GqD9EgQ',
  u'7xqQtHLkKKyMa5T6777OVg',
  u'2vW5tqS1EFel9yYOQ7WZmE',
  u'4CLPNURPcKztF9RRdcWLGP',
  u'6i0YwBLhYs3kkb6qmty0uC',
  u'7GQjJEwqlp6m5HNWVQd6OR',
  u'2yARskwKoNDTbL61y3qqjy',
  u'4HnPZXkx15Y5hfBWh9vtQL',
  u'7zmJyZL3DfYBBYbY8Rve6W',
  u'6leiB1fEsTnVCuPiielde5',
  u'69bp2EbF7Q2rqc5N3ylezZ',
  u'14OxJlLdcHNpgsm4DRwDOB',
  u'1WP1r7fuvRqZRnUaTi2I1Q',
  u'494OU6M7NOf4ICYb4zWCf5',
  u'7cgu4JBW3hq1GwTM1ilkKQ',
  u'0mJbuGt59wWGa5QzxuW0xN',
  u'0RuUVOssl9hBGT95BYKqEI',
  u'2aFiaMXmWsM3Vj72F9ksBl',
  u'0vbbhcA6okLzvsy6WSTlLg',
  u'40YcuQysJ0KlGQTeGUosTC'])

In [169]:
count1=0
count2=0
count3=0
import numpy as np
pidFeatures={}
pidDetails={}
pidSpread={}
for pid in QEPRFOp:
    plFeatures=[]
    plDetails1=[]
    plDetails2=[]
    for track in QEPRFOp[pid]:
        if track in trackFeatures:
            plFeatures.append(trackFeatures[track])
        if track in trackDetails:
            plDetails1.append(trackDetails[track][1])
            plDetails2.append(trackDetails[track][2]&1)
    if len(plFeatures)>0:
        pidFeatures[pid]=[np.average(plFeatures, axis=0), np.min(plFeatures, axis=0), np.max(plFeatures, axis=0), np.median(plFeatures, axis=0)]
        pidSpread[pid]= np.max(plFeatures, axis=0) - np.min(plFeatures, axis=0)
    if len(plDetails1)>0 and len(plDetails2)>0:
        pidDetails[pid]=[min(plDetails1), max(plDetails1), sum(plDetails1)*1.0/len(plDetails1),max(plDetails1)-min(plDetails1),sum(plDetails2)*1.0/len(plDetails2)]

In [170]:
print len(pidFeatures)
print len(pidDetails)
print len(pidSpread)
print len(QEPRFOp)

4500
4500
4500
4500


In [171]:
print pidFeatures[pid]


[array([  5.00000000e+00,   6.47000000e-01,   6.96000000e-02,
         1.24048000e+02,   4.74000000e-02,   3.95000000e-04,
         6.74000000e-01,  -6.10500000e+00,   2.87000000e-01,
         2.03000000e-03]), array([  5.00000000e+00,   6.47000000e-01,   6.96000000e-02,
         1.24048000e+02,   4.74000000e-02,   3.95000000e-04,
         6.74000000e-01,  -6.10500000e+00,   2.87000000e-01,
         2.03000000e-03]), array([  5.00000000e+00,   6.47000000e-01,   6.96000000e-02,
         1.24048000e+02,   4.74000000e-02,   3.95000000e-04,
         6.74000000e-01,  -6.10500000e+00,   2.87000000e-01,
         2.03000000e-03]), array([  5.00000000e+00,   6.47000000e-01,   6.96000000e-02,
         1.24048000e+02,   4.74000000e-02,   3.95000000e-04,
         6.74000000e-01,  -6.10500000e+00,   2.87000000e-01,
         2.03000000e-03])]


In [172]:
plFeatures

[[5, 0.647, 0.0696, 124.048, 0.0474, 0.000395, 0.674, -6.105, 0.287, 0.00203]]

In [173]:
np.max(plFeatures, axis=0)-np.min(plFeatures, axis=0)

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [174]:
TrainFile='./Training/Word2VecBaseFiles/W2VAsFeatures-BigRecallTestingFile750.txt'
with open(TrainFile) as f:
    train = f.readlines()

In [175]:
import sys
def formatFloat(val):
    if val==0:
        return str(0)
    return format(val, '.10f')

def printOp(arr, num):
    count=num
    op=''
    for elem in arr:
        op=op+' '+str(count)+':'+formatFloat(elem)
        count+=1
    return op

In [176]:
newOp=[]
count=0
for l in train:
    pid=l.split()[1].split(':')[1].strip()
    trackid=l.split('#')[1].strip()
    
    
    newL=l.split('#')[0].strip()
    
    if trackid in trackFeatures:
        tfs= trackFeatures[trackid]
        newL=newL+ printOp(tfs,28)
        
        toPrint=[]
        if pid in pidFeatures:
            for arr in pidFeatures[pid]:
                toPrint.append(tfs-arr)

            flist1= np.array(toPrint).flatten()
            newL=newL+ printOp(flist1,38)
    if pid in pidSpread:
        flist2=pidSpread[pid]
        newL=newL+ printOp(flist2,78)
    if pid in pidDetails:
        deets=pidDetails[pid]
        newL=newL+ printOp(deets,88)
    
    if trackid in trackDetails:
        tdeets=[trackDetails[trackid][1],trackDetails[trackid][2]&1]
        flist3=[deets[0]-tdeets[0], deets[1]-tdeets[0], deets[3]-tdeets[0]]
        newL=newL+ printOp(tdeets,93)
        newL=newL+ printOp(flist3,95)
    newL=newL+ ' #'+trackid
    newOp.append(newL)
    count+=1
    if count%1000000==0:
        print count

1000000
2000000
3000000
4000000
5000000
6000000
7000000


In [159]:
newL

'1 qid:1045318 1:1051 2:311.0 3:551 4:1 5:162 6:100 7:62 8:0.4938271605 9:0.4135802469 10:0.0617283951 11:0.0617283951 12:0 13:0 14:0 15:0.0181150000 16:0.1209010000 17:0.0656040000 18:0 19:0 20:0 21:0.0008850000 22:0.0003530000 23:0.0001100000 24:1050 25:1050 26:1050 27:1050 28:10.0000000000 29:0.5820000000 30:0.1530000000 31:126.0750000000 32:0.1030000000 33:0.2900000000 34:0.7980000000 35:-7.3800000000 36:0.7660000000 37:0 38:5.6200000000 39:0.0056400000 40:-0.0234210000 41:7.3171900000 42:0.0193760000 43:-0.0069000600 44:0.2059000000 45:0.0298200000 46:0.3839590000 47:-0.0041439111 48:10.0000000000 49:0.4280000000 50:0.1283000000 51:53.0380000000 52:0.0769000000 53:0.2895570000 54:0.5720000000 55:17.7250000000 56:0.7282000000 57:0 58:-1.0000000000 59:-0.3600000000 60:-0.7150000000 61:-53.8880000000 62:-0.3140000000 63:-0.5960000000 64:-0.0140000000 65:-3.9570000000 66:-0.1950000000 67:-0.1310000000 68:6.0000000000 69:-0.0175000000 70:0.0310000000 71:10.5835000000 72:0.0484000000 73

In [160]:
len(newOp)

14417770

In [163]:
opTrainingFile='./Training/ExternalAndW2VAsFeatures-BigRecall-TestingFile750-2080.txt'
with open(opTrainingFile, 'w') as file_handler:
    for line in newOp:
        file_handler.write(line+'\n')